In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [40]:
import sys
sys.path.append("/home/ec2-user/anaconda3/external/")

# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
import tqdm

In [5]:
PATH = "../Data/kaggle_dog_cat/"
sz=224
bs=128
arch=resnet34

In [58]:
torch.cuda.is_available()
torch.backends.cudnn.enabled

True

In [59]:
m = arch(True)

In [62]:
m

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (b

In [63]:
m = nn.Sequential(*children(m)[:-2], 
                  nn.Conv2d(512, 1024, 3), 
                  nn.AdaptiveAvgPool2d(1), Flatten(),
                  nn.Linear(1024,2),
                  nn.LogSoftmax())

In [64]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)

In [65]:
learn = ConvLearner.from_model_data(m, data)

In [66]:
learn

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d

In [67]:
learn.freeze_to(-5)

In [68]:
m[-5].trainable

True

In [69]:
learn.fit(0.01, 1,best_save_name='resnet_1')

epoch      trn_loss   val_loss   accuracy                     
    0      0.044411   0.026757   0.989579  



[array([0.02676]), 0.9895791583166332]

In [70]:
class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()

In [34]:
x,y = next(iter(data.val_dl))


In [71]:
sfs = SaveFeatures(m[-3])

In [80]:
features = []
outputs = []

#for x,y in iter(data.val_dl):
for x,y in iter(data.trn_dl):
    if len(outputs)>5000:
        break
    py = m(Variable(x.cuda()))
    sfs.remove()
    features_res = to_np(sfs.features)
    y = to_np(y)
    for i in range(len(y)):
        features.append(np.ravel(features_res[i]))
        outputs.append(y[i])

In [81]:
features = np.array(features)
outputs = np.array(outputs)

In [82]:
features.shape

(5120, 1024)

In [76]:
np.save('../Data/kaggle_dog_cat/val_features.npy',features)
np.save('../Data/kaggle_dog_cat/val_outputs.npy',outputs)

In [83]:
np.save('../Data/kaggle_dog_cat/trn_features.npy',features)
np.save('../Data/kaggle_dog_cat/trn_outputs.npy',outputs)

/home/ec2-user/skpca/DNN_for_SKPCA/Data/kaggle_dog_cat
